In [1]:
from fastai.text import *
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.61', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
path = Path('./')

In [7]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-hindi/classification-bbc-news


In [8]:
df_train = pd.read_csv(path/'../../classification_public_datasets/bbc-articles/hi/hi-train.csv', header=None)
df_train.head()

,0,1
0,india,मेट्रो की इस लाइन के चलने से दक्षिणी दिल्ली से...
1,pakistan,नेटिजन यानि इंटरनेट पर सक्रिय नागरिक अब ट्विटर...
2,news,इसमें एक फ़्लाइट एटेनडेंट की मदद की गुहार है औ...
3,india,"प्रतीक खुलेपन का, आज़ाद ख्याली का और भीड़ से अ..."
4,india,ख़ासकर पिछले 10 साल तक प्रधानमंत्री रहे मनमोहन...


In [9]:
df_train = df_train[:-100]
df_valid = df_train[-100:]
df_valid.head()

,0,1
3267,india,मौजूदा मुख्यमंत्री नरेंद्र मोदी के ऊपर इसे तीख...
3268,science,विशेषज्ञों का कहना है कि ये शिशु पैदा न कर पान...
3269,india,यह है मंज़र एक नाटक का जिस में मुंबई की एक गुज...
3270,entertainment,हैरी पॉटर फिल्मों में एमा ने हरमायनी का बेहद ल...
3271,international,ला फ़्लोरिडा में बनाया गया तोतिया रंग का यह ढा...


In [10]:
df_test = pd.read_csv(path/'../../classification_public_datasets/bbc-articles/hi/hi-test.csv', header=None)
df_test.head()

,0,1
0,india,बुधवार को राज्य सभा में विपक्ष के सवालों के जव...
1,india,लखनऊ स्थित पत्रकार समीरात्मज मिश्र को बुलंदशहर...
2,india,लगभग 1300 हेक्टेयर ज़मीन का अधिग्रहण किया जा च...
3,international,हालांकि उनके अंगरक्षकों को बमों को जाम करने वा...
4,india,आयोग का कहना है कि इस तरह के परीक्षण से महिलाओ...


In [11]:
df_train.shape, df_valid.shape, df_test.shape

((3367, 2), (100, 2), (866, 2))

In [12]:
df_train[df_train[1].isnull()]

,0,1


In [13]:
# df_train = df_train.dropna()

In [14]:
df_test[df_test[1].isnull()]

,0,1


In [15]:
# df_test = df_test.dropna()

In [16]:
label_cols = [0]

In [17]:
class HindiTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/hindi/tokenizer/Created from Hindi Wikipedia Articles - 172k/hindi_lm_large.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [18]:
sp = spm.SentencePieceProcessor()
sp.Load('./../../models/hindi/tokenizer/Created from Hindi Wikipedia Articles - 172k/hindi_lm_large.model')
itos = [sp.IdToPiece(int(i)) for i in range(30000)]

In [19]:
itos[:10]

['<unk>', '<s>', '</s>', '▁के', '।', '▁में', '▁है', ',', '▁की', '▁']

In [20]:
# 30,000 is the vocab size that we chose in sentencepiece
hindi_vocab = Vocab(itos)

In [21]:
tokenizer = Tokenizer(tok_func=HindiTokenizer, lang='hi')

In [22]:
tokenizer.special_cases

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep']

In [23]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=hindi_vocab, bs=32)

In [24]:
data_lm.show_batch()

idx,text
0,▁कोई ▁मेट्रो ▁बिना ▁ड्राइवर ▁के ▁चलाई ▁जाएगी . ▁मेट्रो ▁के ▁तीसरे ▁फेज ▁में ▁भारत ▁में ▁पहली ▁बार ▁ड्राइवर लेस ▁तकनीक ▁आएगी ▁लेकिन ▁दुनिया ▁भर ▁में ▁कई ▁देशों ▁में ▁ड्राइवर लेस ▁मेट्रो ▁पहले ▁से ▁ही ▁चलती ▁हैं . ▁इन ▁देशों ▁में ▁ड्राइवर लेस ▁मेट्रो ▁सफल ▁भी ▁रही ▁हैं . ▁दक्षिण ▁कोरिया ▁की ▁राजधानी ▁सोल ▁में ▁ड्राइव लेस ▁ट्रेन ▁कामयाबी ▁से ▁चल ▁रही ▁है . ▁ये ▁मेट्रो ▁ज़मीन ▁के ▁नीचे ▁चलती ▁है
1,"▁कर ▁दिए ▁हैं ▁और ▁वास्तव ▁में ▁उर्दू ▁ने ▁अपने ▁पांव ▁फैला ए ▁हैं ▁और ▁वह ▁नए ▁संचार ▁माध्यमों ▁को ▁बड़ी ▁रास ▁आ ▁रही ▁है . ▁अंग्रेज़ी ▁की ▁तरह ▁इसने ▁भी ▁दूसरी ▁भाषाओं ▁में ▁जगह ▁बनाई ▁और ▁इसके ▁शब्द ▁वहाँ ▁प्रचलित ▁हैं ."" इस ▁फ़िल्म ▁की ▁निर्माता ▁और ▁दिल्ली ▁में ▁जश्न - ए - ब हार ▁के ▁नाम ▁से ▁अंतरराष्ट्रीय ▁मु शायर ों ▁की ▁आयोजक ▁कामना ▁प्रसाद ▁ने ▁कहा , ▁"""
2,▁बड़ी ▁बाधा ▁है ▁कि ▁इन ▁कंपनियों ▁को ▁अब ▁भी ▁बड़े ▁पैमाने ▁पर ▁ऐसे ▁संयंत्र ▁स्थापित ▁करने ▁की ▁ज़रूरत ▁है ▁जो ▁गैस ▁निर्यात ▁को ▁आसान ▁बनाए . इस ▁तरह ▁की ▁आधारभूत ▁संरचनाओं ▁को ▁बनाने ▁में ▁एक ▁लंबा ▁समय ▁लगता ▁है ▁और ▁साल ▁2015 ▁तक ▁ही ▁इनके ▁तैयार ▁होने ▁का ▁अनुमान ▁लगाया ▁जा ▁रहा ▁है . इन में ▁से ▁ज़्यादा तर ▁तो ▁2017 ▁से ▁2019 ▁के ▁बीच ▁ही ▁संचालन ▁के ▁लिए ▁तैयार
3,▁पर ▁क़ब्ज़ ा ▁जमाया . पुरुष ों ▁की ▁56 ▁किलो ▁भार ोत ्तो लन ▁प्रतियोगिता ▁में ▁सुख न ▁डे ▁ने ▁भारत ▁को ▁स्वर्ण ▁पदक ▁दिलवा या . महिला ओं ▁की ▁भार ोत ्तो लन ▁प्रतियोगिता ▁के ▁48 ▁किलो ▁वर्ग ▁में ▁भारत ▁की ▁सं जी ता ▁चा नू ▁खु मुक चा म ▁ने ▁स्वर्ण ▁पदक ▁जीता . पुरुष ों ▁की ▁50 ▁मीटर ▁पि स्टल ▁शूटिंग ▁प्रतियोगिता ▁में ▁भारत ▁के ▁जीत ू
4,"▁36 ▁फ़ीसद ी ▁तक ▁चली ▁जाती ▁है . ▁उ ग्राह ां ▁कहते ▁हैं , ▁"" ज़ ्या दा ▁ब्याज ▁लेने ▁के ▁साथ ▁आ ढ़ ति ए ▁सर चार्ज ▁लगाते ▁हैं , ▁न क़ द ▁पैसा ▁नहीं ▁देते ▁हैं , ▁पर ्ची ▁चलती ▁है . ▁पर ्ची ▁पर ▁मर्ज़ ी ▁के ▁दाम ▁दुकानदार ▁लगाता ▁है . ▁वे ▁दिन ▁पर ▁दिन ▁अमीर ▁होते ▁जा ▁रहे ▁हैं ▁जबकि ▁किसान ▁म कड़ जाल ▁में"


In [25]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [26]:
# Loading the pretrained language model on hindi wikipedia
learn.load('../../../models/hindi/lm/ULMFiT - Trained on Hindi Wikipedia Articles - 172k/model')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (3367 items)
x: LMTextList
▁x x bo s ▁मेट्रो ▁की ▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी ▁दिल्ली ▁से ▁नोएडा ▁जाने ▁का ▁समय ▁काफी ▁कम ▁हो ▁जाएगा ▁और ▁यात्रियों ▁को ▁राजीव ▁चौक ▁या ▁मंडी ▁हाउस ▁से ▁होकर ▁नहीं ▁जाना ▁पड़ेगा . लेकिन , ▁यह ▁म जेंट ा ▁लाइन ▁इसलिए ▁भी ▁महत्वपूर्ण ▁है ▁क्योंकि ▁इस ▁पर ▁ड्राइव लै स ▁यानी ▁बिना ▁ड्राइवर ▁वाली ▁मेट्रो ▁चलाने ▁की ▁योजना ▁है . ▁ऐसा ▁भारत ▁में ▁पहली ▁बार ▁होगा ▁जब ▁कोई ▁मेट्रो ▁बिना ▁ड्राइवर ▁के ▁चलाई ▁जाएगी . ▁मेट्रो ▁के ▁तीसरे ▁फेज ▁में ▁भारत ▁में ▁पहली ▁बार ▁ड्राइवर लेस ▁तकनीक ▁आएगी ▁लेकिन ▁दुनिया ▁भर ▁में ▁कई ▁देशों ▁में ▁ड्राइवर लेस ▁मेट्रो ▁पहले ▁से ▁ही ▁चलती ▁हैं . ▁इन ▁देशों ▁में ▁ड्राइवर लेस ▁मेट्रो ▁सफल ▁भी ▁रही ▁हैं . ▁दक्षिण ▁कोरिया ▁की ▁राजधानी ▁सोल ▁में ▁ड्राइव लेस ▁ट्रेन ▁कामयाबी ▁से ▁चल ▁रही ▁है . ▁ये ▁मेट्रो ▁ज़मीन ▁के ▁नीचे ▁चलती ▁है ▁इसमें ▁ड्राइवर ▁का ▁के बिन ▁भी ▁नहीं ▁होता ▁है . ▁यूरोप ▁में ▁डेनमार्क , ▁स्पेन , ▁इटली , ▁फ्रांस , ▁जर्मनी , ▁हंगरी , ▁स्विट्जरलैंड ▁और ▁ब्रिटेन ▁में 

In [27]:
# Fine tuning the prtrained LM on current dataset

In [28]:
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,time


LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [29]:
learn.recorder.plot()

In [30]:
learn.freeze()

In [31]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.105361,3.697194,0.338742,00:51


In [32]:
learn.save('fit_head', with_opt=True)

In [33]:
learn.load('fit_head', with_opt=True);

In [34]:
learn.unfreeze()

In [35]:
learn.fit_one_cycle(5, 1e-3, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,3.658858,3.476438,0.366434,01:02
1,3.554282,3.154902,0.409556,01:02
2,3.260366,2.833583,0.460463,01:03
3,3.003796,2.622506,0.494727,01:02
4,2.833259,2.558339,0.504660,01:02


In [36]:
learn.save('fine_tuned', with_opt=True)

In [37]:
learn.load('fine_tuned', with_opt=True);

In [38]:
learn.predict('▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी ',n_words=10)

'▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी  श िल टी म ▁की ▁स्थिति ▁बेहतर ▁होगी ▁और ▁किसी'

In [39]:
learn.save_encoder('fine_tuned_enc')

In [40]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=hindi_vocab)

In [41]:
data_clas.show_batch()

text,target
▁x x bo s ▁1920 ▁के ▁दशक ▁में ▁फ़ी फ़ा ▁के ▁अध्यक्ष ▁जूल ्स ▁रि मे ▁और ▁फ़्रांस ▁के ▁फ़ुटबॉल ▁प्रशासकों ▁ने ▁दुनिया ▁की ▁बेहतरीन ▁फ़ुटबॉल ▁टीम ▁तय ▁करने ▁के ▁लिए ▁प्रतियोगिता ▁कराने ▁का ▁विचार ▁किया . ▁1929 ▁में ▁फ़ी फ़ा ▁ने ▁एक ▁प्रस्ताव ▁पारित ▁करके ▁विश्व ▁कप ▁फ़ुटबॉल ▁आयोजित ▁कराने ▁का ▁फ़ैसला ▁किया . 19 30 ▁में ▁शुरू ▁हुई ▁ये ▁प्रतियोगिता ▁हर ▁चार ▁साल ▁पर ▁आयोजित ▁होती ▁है .,sport
"▁x x bo s ▁खिलाड़ियों ▁ने ▁रंगीन ▁पोशाक ें ▁पहन नी ▁शुरू ▁की , ▁गेंद ▁उ जल ी ▁हो ▁गई ▁और ▁अंपायर ▁तक ▁रंगीन ▁पोशाक ▁में ▁नज़र ▁आने ▁लगे . ▁मैच ▁दिन - रात ▁के ▁होने ▁लगे ▁और ▁50 -50 ▁ओवर ों ▁के ▁मैच ▁होने ▁लगे . पहले ▁तीन ▁विश्व ▁कप ▁इंग्लैंड ▁में ▁आयोजित ▁हुए ▁और ▁तीन ▁में ▁दो ▁विश्व ▁कप ▁का ▁ख़िताब ▁वेस्टइंडीज ़ ▁ने ▁जीता . ▁लेकिन ▁1983",sport
▁x x bo s ▁शुभ ्र ांशु ▁चौधरी ▁ने ▁सात ▁वर्षों ▁के ▁लंबे ▁शोध ▁के ▁बाद ▁ये ▁किताब ▁लिखी ▁है . अंग्रेज़ी ▁और ▁हिंदी ▁दोनों ▁भाषाओं ▁में ▁प्रकाशित ▁इस ▁किताब ▁का ▁एक ▁अंश ▁हम ▁अपने ▁पाठकों ▁के ▁लिए ▁प्रकाशित ▁कर ▁रहे ▁हैं . समय ▁जैसे ▁उड़ ▁गया . ▁फिर ▁विदा ▁लेने ▁का ▁समय ▁हो ▁गया . ▁कैंप ▁में ▁सभी ▁कतार ▁बांध ▁कर ▁हाथ ▁मिलाने ▁और ▁लाल ▁सलाम ▁कहने ▁को ▁खड़े,india
▁x x bo s ▁1. ▁यात्रा ▁का ▁आनंद ः ▁दत्तात्रेय ▁बालकृष्ण ▁' का का ' ▁कालेलकर का का ▁कालेलकर ▁शुरू ▁में ▁गुजराती ▁में ▁लिखते ▁थे . ▁उनकी ▁' हिमालय ▁यात्रा ' ▁अनु दित ▁होकर ▁हिंदी ▁में ▁छपी ▁थी . ▁बाद ▁में ▁वो ▁हिंदी ▁में ▁ही ▁लिखने ▁लगे . उन्हें ▁यात्राओं ▁में ▁बहुत ▁आनंद ▁आता ▁था . ▁देश ▁और ▁विदेश ▁में ▁अनेक ▁प्रवास ▁उन्होंने ▁लिपि बद्ध ▁किए . ▁भारतीय ▁सांस्कृतिक ▁संबंध,india
▁x x bo s ▁गोरखा ▁जन मुक्ति ▁मोर्चा ▁की ▁अपील ▁पर ▁जारी ▁बे मिया दी ▁बंद ▁के ▁विरोध ▁में ▁आवाज़ ें ▁तो ▁पहले ▁से ▁ही ▁उठ ▁रही ▁थीं . ▁लेकिन ▁अब ▁विरोध ▁के ▁स्वर ▁तेज़ ▁होने ▁लगे ▁हैं . दिल्ली ▁में ▁केंद्रीय ▁गृह ▁मंत्री ▁राज नाथ ▁सिंह ▁से ▁गोरखा लैंड ▁मांग ▁समन्वय ▁समिति ▁के ▁प्रतिनिधिमंडल ▁की ▁मुलाक़ात ▁और ▁उसके ▁बाद ▁25 ▁दिनों ▁से ▁जारी ▁आम रण ▁अनशन ▁वापस ▁लेने ▁से,india


In [42]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)

In [43]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (3367 items)
x: TextList
▁x x bo s ▁मेट्रो ▁की ▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी ▁दिल्ली ▁से ▁नोएडा ▁जाने ▁का ▁समय ▁काफी ▁कम ▁हो ▁जाएगा ▁और ▁यात्रियों ▁को ▁राजीव ▁चौक ▁या ▁मंडी ▁हाउस ▁से ▁होकर ▁नहीं ▁जाना ▁पड़ेगा . लेकिन , ▁यह ▁म जेंट ा ▁लाइन ▁इसलिए ▁भी ▁महत्वपूर्ण ▁है ▁क्योंकि ▁इस ▁पर ▁ड्राइव लै स ▁यानी ▁बिना ▁ड्राइवर ▁वाली ▁मेट्रो ▁चलाने ▁की ▁योजना ▁है . ▁ऐसा ▁भारत ▁में ▁पहली ▁बार ▁होगा ▁जब ▁कोई ▁मेट्रो ▁बिना ▁ड्राइवर ▁के ▁चलाई ▁जाएगी . ▁मेट्रो ▁के ▁तीसरे ▁फेज ▁में ▁भारत ▁में ▁पहली ▁बार ▁ड्राइवर लेस ▁तकनीक ▁आएगी ▁लेकिन ▁दुनिया ▁भर ▁में ▁कई ▁देशों ▁में ▁ड्राइवर लेस ▁मेट्रो ▁पहले ▁से ▁ही ▁चलती ▁हैं . ▁इन ▁देशों ▁में ▁ड्राइवर लेस ▁मेट्रो ▁सफल ▁भी ▁रही ▁हैं . ▁दक्षिण ▁कोरिया ▁की ▁राजधानी ▁सोल ▁में ▁ड्राइव लेस ▁ट्रेन ▁कामयाबी ▁से ▁चल ▁रही ▁है . ▁ये ▁मेट्रो ▁ज़मीन ▁के ▁नीचे ▁चलती ▁है ▁इसमें ▁ड्राइवर ▁का ▁के बिन ▁भी ▁नहीं ▁होता ▁है . ▁यूरोप ▁में ▁डेनमार्क , ▁स्पेन , ▁इटली , ▁फ्रांस , ▁जर्मनी , ▁हंगरी , ▁स्विट्जरलैंड ▁और ▁ब्रिटेन ▁में ▁भी ▁

In [44]:
learn.freeze()

In [45]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


In [46]:
learn.recorder.plot()

In [47]:
learn.loss_func.func

CrossEntropyLoss()

In [48]:
mcc = MatthewsCorreff()

In [49]:
learn.metrics = [mcc, accuracy]

In [50]:
learn.fit_one_cycle(1, 1e-1, moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,1.051857,0.682144,0.691980,0.760000,00:18


In [51]:
learn.save('first-full')

In [52]:
learn.load('first-full');

In [53]:
learn.freeze_to(-2)
learn.fit_one_cycle(5, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.679242,0.454796,0.819732,0.860000,00:22
1,0.653243,0.381277,0.832455,0.870000,00:23
2,0.576620,0.341486,0.844997,0.880000,00:21
3,0.458931,0.225752,0.886267,0.910000,00:25
4,0.372466,0.198024,0.897105,0.920000,00:23


In [54]:
learn.save('second-full')

In [55]:
learn.load('second-full')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (3367 items)
x: TextList
▁x x bo s ▁मेट्रो ▁की ▁इस ▁लाइन ▁के ▁चलने ▁से ▁दक्षिणी ▁दिल्ली ▁से ▁नोएडा ▁जाने ▁का ▁समय ▁काफी ▁कम ▁हो ▁जाएगा ▁और ▁यात्रियों ▁को ▁राजीव ▁चौक ▁या ▁मंडी ▁हाउस ▁से ▁होकर ▁नहीं ▁जाना ▁पड़ेगा . लेकिन , ▁यह ▁म जेंट ा ▁लाइन ▁इसलिए ▁भी ▁महत्वपूर्ण ▁है ▁क्योंकि ▁इस ▁पर ▁ड्राइव लै स ▁यानी ▁बिना ▁ड्राइवर ▁वाली ▁मेट्रो ▁चलाने ▁की ▁योजना ▁है . ▁ऐसा ▁भारत ▁में ▁पहली ▁बार ▁होगा ▁जब ▁कोई ▁मेट्रो ▁बिना ▁ड्राइवर ▁के ▁चलाई ▁जाएगी . ▁मेट्रो ▁के ▁तीसरे ▁फेज ▁में ▁भारत ▁में ▁पहली ▁बार ▁ड्राइवर लेस ▁तकनीक ▁आएगी ▁लेकिन ▁दुनिया ▁भर ▁में ▁कई ▁देशों ▁में ▁ड्राइवर लेस ▁मेट्रो ▁पहले ▁से ▁ही ▁चलती ▁हैं . ▁इन ▁देशों ▁में ▁ड्राइवर लेस ▁मेट्रो ▁सफल ▁भी ▁रही ▁हैं . ▁दक्षिण ▁कोरिया ▁की ▁राजधानी ▁सोल ▁में ▁ड्राइव लेस ▁ट्रेन ▁कामयाबी ▁से ▁चल ▁रही ▁है . ▁ये ▁मेट्रो ▁ज़मीन ▁के ▁नीचे ▁चलती ▁है ▁इसमें ▁ड्राइवर ▁का ▁के बिन ▁भी ▁नहीं ▁होता ▁है . ▁यूरोप ▁में ▁डेनमार्क , ▁स्पेन , ▁इटली , ▁फ्रांस , ▁जर्मनी , ▁हंगरी , ▁स्विट्जरलैंड ▁और ▁ब्रिटेन ▁में ▁भी ▁

In [56]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.328369,0.186278,0.897105,0.920000,00:41
1,0.314577,0.150040,0.948743,0.960000,00:37
2,0.292240,0.123170,0.936069,0.950000,00:42
3,0.268563,0.113161,0.961727,0.970000,00:39
4,0.254961,0.105568,0.948743,0.960000,00:43


In [57]:
learn.save('final')

In [58]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
for index, row in df_result.iterrows():
    pred = learn.predict(data_clas.test_ds[index])
    for node in all_nodes:
        row[node] = pred[2][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[pred[0].data.item()]
df_result.head()

,query,actual_label,predicted_label,business,international,social,multimedia,sport,news,entertainment,learningenglish,pakistan,science,china,india,institutional,southasia
0,बुधवार को राज्य सभा में विपक्ष के सवालों के जव...,india,india,0.0515407,0.00113703,0.000351522,9.15776e-05,0.000574766,0.0149112,0.000132131,3.69353e-05,0.00020287,0.000607966,0.00184311,0.922291,0.000217157,0.00606259
1,लखनऊ स्थित पत्रकार समीरात्मज मिश्र को बुलंदशहर...,india,india,5.99e-06,0.000144832,0.000209561,0.000128753,9.05685e-05,0.000115433,0.000449393,2.07739e-06,1.66698e-06,2.60421e-05,1.06033e-05,0.998796,1.18443e-05,7.65474e-06
2,लगभग 1300 हेक्टेयर ज़मीन का अधिग्रहण किया जा च...,india,india,0.0246157,0.00421849,0.000393538,0.000203698,0.00019923,0.0209263,0.000457686,6.0754e-05,0.000126208,0.00366882,0.00123746,0.942905,0.000108416,0.000878674
3,हालांकि उनके अंगरक्षकों को बमों को जाम करने वा...,international,international,6.81034e-05,0.970229,0.00158513,0.00507487,0.000142925,0.000648952,0.000225849,0.00027427,3.72073e-05,0.0172274,0.000435269,0.00346815,0.000549893,3.32306e-05
4,आयोग का कहना है कि इस तरह के परीक्षण से महिलाओ...,india,international,0.000164305,0.800239,0.0129369,0.0058761,0.00183652,0.0120559,0.00986674,0.000868305,0.00236773,0.0150169,0.00112251,0.134176,0.00286304,0.000610023


In [59]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.7875288683602771

In [60]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.7161765395964725

In [61]:
df_result.to_csv('bbc_articles_result.csv', index=False)